In [37]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
import pprint
import seaborn as sns
import CMR_IA as cmr
import scipy as sp
sns.set_context('paper')
np.set_printoptions(suppress=True)
# pd.set_option('display.max_columns', None)

In [38]:
def simu_success(tag, params):
    # which task
    if tag == "Asso-CR":
        path = "../Data/simuS1_group1_design.pkl"
        nitems = 96
        test1_num = 40
    elif tag == "Pair-CR":
        path = "../Data/simuS1_group2_design.pkl"
        nitems = 176
        test1_num = 80
    elif tag == "Item-CR":
        path = "../Data/simuS1_group3_design.pkl"
        nitems = 136
        test1_num = 80

    # load stimuli
    with open(path, 'rb') as inp:
        df_study = pickle.load(inp)
        df_test = pickle.load(inp)
    # df_study = df_study.loc[df_study.session < 200]
    # df_test = df_test.loc[df_test.session < 200]

    # load semantic matrix
    s_mat = np.load('../Data/wordpools/ltp_FR_similarity_matrix.npy')

    # update n_item in params to fit the study
    params.update(nitems_in_accumulator = nitems)
    # print(params)

    # run CMR
    df_simu, f_in_acc, f_in_dif = cmr.run_success_multi_sess(params, df_study, df_test ,s_mat)
    df_simu['test'] = df_test['test']
    df_simu = df_simu.merge(df_test,on=['session','test','test_itemno1','test_itemno2'])

    # get f_in
    sessions = np.unique(df_simu.session)
    tmp_corr_fin = []
    tmp_omax_fin = []

    for sess in sessions:
        df_tmp = df_study.loc[df_study.session == sess]
        tmp1 = df_tmp.study_itemno1.to_numpy()
        tmp2 = df_tmp.study_itemno2.to_numpy()
        df_tmp2 = df_test.loc[df_test.session == sess]
        tmp3 = df_tmp2.test_itemno1[df_tmp2.test_itemno1 >= 0].to_numpy()
        tmp4 = df_tmp2.test_itemno2[df_tmp2.test_itemno2 >= 0].to_numpy()
        tmp = np.concatenate((tmp1, tmp2, tmp3, tmp4))
        tmp = np.unique(tmp)  # sort

        tmp_corr = df_simu.loc[df_simu.session == sess,"correct_ans"][test1_num:]
        corrid = np.searchsorted(tmp, tmp_corr)

        corr_fin = [f_in_dif[sess][i][id] for i, id in enumerate(corrid)]
        omax_fin = [np.max(np.delete(f_in_dif[sess][i], id)) for i, id in enumerate(corrid)]

        tmp_corr_fin = tmp_corr_fin + [-1] * test1_num + corr_fin
        tmp_omax_fin = tmp_omax_fin + [-1] * test1_num + omax_fin

    df_simu['corr_fin'] = tmp_corr_fin
    df_simu['omax_fin'] = tmp_omax_fin

    # optimal threshold for test1
    csim_two = df_simu.query("test==1").groupby("correct_ans").csim.mean()
    opt_thresh = np.mean(csim_two)
    df_simu['s_resp'] = df_simu.apply(lambda x: (1 if x['csim'] > opt_thresh else 0) if x['test'] == 1 else x['s_resp'], axis=1)

    return df_simu

In [39]:
def anal_perform(df_simu):

    # get correctness
    df_simu['correct'] = df_simu.s_resp == df_simu.correct_ans

    # recognition performance
    df_recog = df_simu.query("test==1")
    hr_far = df_recog.groupby("correct_ans")["s_resp"].mean().to_frame(name="Yes rate")
    hr = hr_far['Yes rate'][0]
    far = hr_far['Yes rate'][1]
    z_hr_far = sp.stats.norm.ppf(hr_far)
    d_prime = z_hr_far[1].item() - z_hr_far[0].item()
    print("recognition: \n", hr_far)
    print("d_prime: ", d_prime)

    # cued recall performance
    df_cr = df_simu.query("test==2")
    p_rc = df_cr.correct.mean()
    print("cued recall: \n", p_rc)

    # analyze pair
    def get_pair(df_tmp):
        df_tmp_pair = pd.pivot_table(df_tmp,index="pair_idx",columns="test",values="correct")
        df_tmp_pair.columns = ["test1","test2"]
        df_tmp_pair.reset_index(inplace=True)
        return df_tmp_pair

    df_simu_p = df_simu.query("pair_idx >= 0")
    df_pair = df_simu_p.groupby("session").apply(get_pair).reset_index()
    df_tab = pd.crosstab(index=df_pair.test2,columns=df_pair.test1,normalize=False)
    print("contingency table: \n", df_tab)
    print("contingency table norm: \n", pd.crosstab(index=df_pair.test2,columns=df_pair.test1,normalize='all'))

    # compute" Q
    def Yule_Q(A, B, C, D):
        return (A * D - B * C) / (A * D + B * C)
    q = Yule_Q(df_tab[1][1]+0.5,df_tab[0][1]+0.5,df_tab[1][0]+0.5,df_tab[0][0]+0.5)  # add 0.5
    print("Q: ", q)

    return p_rc, hr, far, d_prime, q

### Study Params

In [40]:
# study params should be the same across three groups
params = cmr.make_default_params()
params.update(
    beta_enc = 0.3,
    gamma_fc = 0.2,
    gamma_cf = 0.2,
    s_fc = 0.2,
    s_cf = 0.2,
    phi_s = 1,
    phi_d = 0.6,
    d_ass = 1,
    learn_while_retrieving = False,
    var_enc = 1,
    bad_enc_ratio = 1,
)

In [41]:
params.update(
    beta_cue = 0.4,
    beta_rec = 0.1,  # beta for retrieved item
    beta_distract = 0.1,
    # beta_rec_post = 0.1,
    c_thresh = 0.01,
    c_thresh_ass = 0.5,
    kappa = 0.01,
    lamb = 0.002,
    eta = 0.002,
    omega = 3,
    alpha = 1
)
params

{'beta_enc': 0.3,
 'beta_rec': 0.1,
 'beta_cue': 0.4,
 'beta_rec_post': 0.5,
 'beta_distract': 0.1,
 'phi_s': 1,
 'phi_d': 0.6,
 's_cf': 0.2,
 's_fc': 0.2,
 'kappa': 0.01,
 'eta': 0.002,
 'omega': 3,
 'alpha': 1,
 'c_thresh': 0.01,
 'c_thresh_ass': 0.5,
 'd_ass': 1,
 'lamb': 0.002,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 50,
 'max_recalls': 50,
 'learn_while_retrieving': False,
 'a': 2800,
 'b': 20,
 'm': 0,
 'n': 1,
 'c1': 0,
 'No_recall': None,
 'var_enc': 1,
 'bad_enc_ratio': 1,
 'gamma_fc': 0.2,
 'gamma_cf': 0.2}

### Association - CR (Group1)

In [42]:
rng = np.random.default_rng()

In [43]:
tag = "Asso-CR"
df_simu_g1 = simu_success(tag, params)

CMR2 Time: 68.6192991733551


In [44]:
n_subj = 30
n_per_subj = 3
g1_sesses = rng.choice(np.arange(1000),n_subj*n_per_subj,replace=False)

In [45]:
g1_stats = []
for i in range(n_subj):
    this_sesses = g1_sesses[n_per_subj*i:n_per_subj*(i+1)]
    this_df = df_simu_g1.loc[np.isin(df_simu_g1.session, this_sesses)].copy()    
    p_rc, hr, far, d_prime, q = anal_perform(this_df)
    g1_stats.append([p_rc, hr, far, d_prime, q])

recognition: 
              Yes rate
correct_ans          
0            0.100000
1            0.916667
d_prime:  2.6645456926452384
cued recall: 
 0.4166666666666667
contingency table: 
 test1  0.0  1.0
test2          
0.0      5   30
1.0      0   25
contingency table norm: 
 test1       0.0       1.0
test2                    
0.0    0.083333  0.500000
1.0    0.000000  0.416667
Q:  0.8038585209003215
recognition: 
              Yes rate
correct_ans          
0            0.183333
1            0.850000
d_prime:  1.9391681811376547
cued recall: 
 0.6166666666666667
contingency table: 
 test1  0.0  1.0
test2          
0.0      8   15
1.0      1   36
contingency table norm: 
 test1       0.0   1.0
test2                
0.0    0.133333  0.25
1.0    0.016667  0.60
Q:  0.8605697151424287
recognition: 
              Yes rate
correct_ans          
0            0.183333
1            0.866667
d_prime:  2.01350640828065
cued recall: 
 0.4666666666666667
contingency table: 
 test1  0.0  1.0
test2  

In [46]:
print(np.array(g1_stats))
print(np.mean(np.array(g1_stats),axis=0))
print(sp.stats.sem(np.array(g1_stats),axis=0))

[[0.41666667 0.1        0.91666667 2.66454569 0.80385852]
 [0.61666667 0.18333333 0.85       1.93916818 0.86056972]
 [0.46666667 0.18333333 0.86666667 2.01350641 0.90373281]
 [0.4        0.2        0.85       1.87805462 0.64742268]
 [0.33333333 0.23333333 0.86666667 1.83868491 0.48854962]
 [0.41666667 0.18333333 0.9        2.18428636 0.49307479]
 [0.4        0.15       0.85       2.07286678 0.39175258]
 [0.4        0.23333333 0.88333333 1.91972946 0.85138539]
 [0.4        0.21666667 0.76666667 1.51141367 0.93860846]
 [0.45       0.2        0.88333333 2.03343741 0.87927107]
 [0.45       0.2        0.91666667 2.22461536 0.82779456]
 [0.45       0.18333333 0.96666667 2.73664943 0.62721893]
 [0.38333333 0.13333333 0.9        2.39232318 0.81305638]
 [0.46666667 0.18333333 0.81666667 1.80546958 0.79069767]
 [0.41666667 0.18333333 0.8        1.74435603 0.76923077]
 [0.38333333 0.13333333 0.83333333 2.07819318 0.42495127]
 [0.51666667 0.1        0.95       2.92640519 0.29787234]
 [0.6        0

### Pair - CR (Group2)

In [47]:
tag = "Pair-CR"
df_simu_g2 = simu_success(tag, params)

CMR2 Time: 146.84375882148743


In [48]:
n_subj = 30
n_per_subj = 5
g2_sesses = rng.choice(np.arange(1000),n_subj*n_per_subj,replace=False)

In [49]:
g2_stats = []
for i in range(n_subj):
    this_sesses = g2_sesses[n_per_subj*i:n_per_subj*(i+1)]
    this_df = df_simu_g2.loc[np.isin(df_simu_g2.session, this_sesses)].copy()    
    p_rc, hr, far, d_prime, q = anal_perform(this_df)
    g2_stats.append([p_rc, hr, far, d_prime, q])

recognition: 
              Yes rate
correct_ans          
0                0.01
1                0.95
d_prime:  3.971201500992313
cued recall: 
 0.44
contingency table: 
 test1  0.0  1.0
test2          
0.0     10  102
1.0      0   88
contingency table norm: 
 test1   0.0   1.0
test2            
0.0    0.05  0.51
1.0    0.00  0.44
Q:  0.8954614992350841
recognition: 
              Yes rate
correct_ans          
0               0.005
1               0.980
d_prime:  4.629578214180723
cued recall: 
 0.43
contingency table: 
 test1  0.0  1.0
test2          
0.0      4  110
1.0      0   86
contingency table norm: 
 test1   0.0   1.0
test2            
0.0    0.02  0.55
1.0    0.00  0.43
Q:  0.7514060742407199
recognition: 
              Yes rate
correct_ans          
0                0.04
1                0.98
d_prime:  3.8044349818839924
cued recall: 
 0.41
contingency table: 
 test1  0.0  1.0
test2          
0.0      4  114
1.0      0   82
contingency table norm: 
 test1   0.0   1.0
test2

In [50]:
print(np.array(g2_stats))
print(np.mean(np.array(g2_stats),axis=0))
print(sp.stats.sem(np.array(g2_stats),axis=0))

[[ 0.44        0.01        0.95        3.9712015   0.8954615 ]
 [ 0.43        0.005       0.98        4.62957821  0.75140607]
 [ 0.41        0.04        0.98        3.80443498  0.7327888 ]
 [ 0.41        0.015       0.97        4.05088399  0.81012658]
 [ 0.435       0.03        0.975       3.84075759  0.40650407]
 [ 0.415       0.025       0.965       3.77187466  0.52452026]
 [ 0.31        0.025       0.985       4.13005436  0.52705061]
 [ 0.385       0.025       0.98        4.0137129   0.70746634]
 [ 0.4         0.005       0.965       4.38773998  0.82816048]
 [ 0.37        0.035       0.96        3.56259674  0.82888087]
 [ 0.385       0.          0.98               inf  0.70746634]
 [ 0.42        0.04        0.965       3.56259674  0.84095861]
 [ 0.375       0.02        0.99        4.38009678  0.50698603]
 [ 0.435       0.015       0.98        4.22383929  0.29242263]
 [ 0.485       0.01        0.985       4.49643825  0.74329502]
 [ 0.365       0.025       0.975       3.91992797  0.27

d:\Program Files\Anaconda3\envs\CMR_IA\lib\site-packages\numpy\core\_methods.py:236: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


### Item - CR (Group3)

In [51]:
tag = "Item-CR"
df_simu_g3 = simu_success(tag, params)

CMR2 Time: 120.63800477981567


In [52]:
n_subj = 30
n_per_subj = 4
g3_sesses = rng.choice(np.arange(1000),n_subj*n_per_subj,replace=False)

In [53]:
g3_stats = []
for i in range(n_subj):
    this_sesses = g3_sesses[n_per_subj*i:n_per_subj*(i+1)]
    this_df = df_simu_g3.loc[np.isin(df_simu_g3.session, this_sesses)].copy()    
    p_rc, hr, far, d_prime, q = anal_perform(this_df)
    g3_stats.append([p_rc, hr, far, d_prime, q])

recognition: 
              Yes rate
correct_ans          
0             0.17500
1             0.81875
d_prime:  1.8452009979459503
cued recall: 
 0.375
contingency table: 
 test1  0.0  1.0
test2          
0.0     29   71
1.0      0   60
contingency table norm: 
 test1      0.0      1.0
test2                  
0.0    0.18125  0.44375
1.0    0.00000  0.37500
Q:  0.9607250755287009
recognition: 
              Yes rate
correct_ans          
0             0.19375
1             0.88125
d_prime:  2.045418625295348
cued recall: 
 0.48125
contingency table: 
 test1  0.0  1.0
test2          
0.0     14   69
1.0      5   72
contingency table norm: 
 test1      0.0      1.0
test2                  
0.0    0.08750  0.43125
1.0    0.03125  0.45000
Q:  0.46668991977677016
recognition: 
              Yes rate
correct_ans          
0               0.175
1               0.825
d_prime:  1.8691785821469602
cued recall: 
 0.39375
contingency table: 
 test1  0.0  1.0
test2          
0.0     25   72
1.0     

In [54]:
print(np.array(g3_stats))
print(np.mean(np.array(g3_stats),axis=0))
print(sp.stats.sem(np.array(g3_stats),axis=0))

[[0.375      0.175      0.81875    1.845201   0.96072508]
 [0.48125    0.19375    0.88125    2.04541863 0.46668992]
 [0.39375    0.175      0.825      1.86917858 0.71750626]
 [0.38125    0.1875     0.81875    1.79775827 0.62781016]
 [0.40625    0.2125     0.76875    1.53251332 0.63440157]
 [0.36875    0.15625    0.84375    2.01998034 0.64085189]
 [0.4375     0.225      0.775      1.51083005 0.78284182]
 [0.38125    0.15       0.775      1.79184842 0.72036824]
 [0.40625    0.1875     0.83125    1.84626318 0.45080805]
 [0.39375    0.21875    0.8125     1.66356832 0.81962107]
 [0.5125     0.2        0.8125     1.72876779 0.79602849]
 [0.46875    0.18125    0.75       1.58510146 0.65659807]
 [0.46875    0.2375     0.81875    1.62497915 0.6724764 ]
 [0.3375     0.18125    0.7875     1.70838855 0.6346511 ]
 [0.4        0.1875     0.78125    1.66356832 0.73210634]
 [0.38125    0.24375    0.81875    1.60490228 0.80055021]
 [0.48125    0.1375     0.825      2.02620966 0.52758881]
 [0.4875     0